In [1]:
import h5py
import numpy as np
import pandas as pd
import os
import gc

In [2]:
def convert(filename, start, end):
    f = h5py.File(filename, 'r')
    data = f['Data']['Table Layout'][start:end]
    d = {"ut_time" : data['ut1_unix'], "gps_site" : data['gps_site'], "sat_id" : data['sat_id'], "gdlat" : data['gdlat'], "glon" : data['glon'], "tec" : data['tec'], "dtec" : data['dtec'], "elm" : data['elm']}
    print("dict created from dataset")
    del data
    gc.collect()
    df = pd.DataFrame(d)
    print("dataframe created from dict")
    del d
    gc.collect()
    df['datetime'] = pd.to_datetime(df.ut_time, unit='s')
    del df['ut_time']
    df['site'] = df.gps_site.str.decode('utf-8')
    del df['gps_site']
    #df['vtec'] = df.tec * np.sqrt(1 - ((6371 * np.cos(np.deg2rad(df.elm)) / 6721)**2))
    df['log_tec'] = np.log10(df['tec'])
    #df['log_vtec'] = np.log10(df['vtec'])
    return df

def quick_convert(filename, start, end):
    f = h5py.File(filename, 'r')
    data = f['Data']['Table Layout'][start:end]
    d = {"ut_time" : data['ut1_unix'], "gdlat" : data['gdlat'], "glon" : data['glon'], "tec" : data['tec'],  "dtec" : data['dlos_tec'], "elm" : data['elm'], "gps_site" : data['gps_site'], "sat_id" : data['sat_id']}
    print("dict created from dataset")
    del data
    gc.collect()
    print("dataset deleted")
    df = pd.DataFrame(d)
    print("dataframe created from dict")
    del d
    gc.collect()
    print("dict deleted")
    df['datetime'] = pd.to_datetime(df.ut_time, unit='s')
    del df['ut_time']
    return df.round(2)

In [5]:
filename = 'los_20170907.004.h5'
f = h5py.File(filename, 'r')
data = f['Data']['Table Layout']
print(data.size / 2)

106653907.5


In [8]:

df = quick_convert(filename, 106653907, -1)

dict created from dataset
dataset deleted
dataframe created from dict
dict deleted


In [9]:
print(df.head())

   gdlat    glon    tec  dtec    elm gps_site  sat_id   datetime
0 -10.20  130.22  24.00  0.24  51.60  b'00na'       1 2017-09-07
1 -14.19  131.54  18.47  0.49  57.96  b'00na'       3 2017-09-07
2 -14.75  135.19  20.04  0.93  30.07  b'00na'       5 2017-09-07
3 -17.95  130.46  16.11  0.29  26.46  b'00na'       6 2017-09-07
4  -9.99  128.15  23.24  0.20  38.46  b'00na'       7 2017-09-07


In [1]:
df.to_pickle(filename[0:12] + '.pkl')

NameError: name 'df' is not defined

In [6]:
data = f['Data']['Table Layout'][0:10]

In [11]:
data['dlos_tec']

array([1.496175, 0.216619, 0.934936, 0.204711, 0.281104, 0.192094,
       0.633529, 0.283237, 0.956732, 0.362169])